# Autograd : Automatic Diffentitation
    
Pytorch에서 모든 neural network들의 핵심은 `autograd` package이다. 
간단하게 이를 확인하고 neural network를 학습해보자.  

<br/>

`autograd` package는 Tensor들의 모든 operation들을 위한 automatic differentiation을 제공한다.  
`autograd`는 `define-by-run` framework이고, 이게 의미하는 것은 코드 실행 방법에 따라 backprob이 정의되며,  
각 iteration마다 backprop이 다를 수 있다는 것이다.

<br/>

조금 더 쉬운 용어와 예제를 이용해서 확인해보자  
    

## Tensor
    
`torch.Tensor`는 package의 핵심 class이다. 만약 `requires_grad`라는 attribute를 True로 설정하면, 모든 operation들을 추적하기 시작한다. computation이 완료되었을 때, `.backward()`라는 method를 call할 수 있고, 그 결과 자동적으로 계산된 모든 gradients 갖게된다. 해당 tensor를 위한 gradient는 `.grad`라는 attribute에 누적된다.

<br/>

tensor가 history를 추적하는 것을 멈추게 하려면, `.detach()`를 호출하면 된다. `.detach()` method는 computation history를 tensor와 분리시키고, 앞으로 computation이 추적되는 것을 방지한다.

<br/>

hisotry를 추적하는 것을 막기 위해서(메모리를 사용하는 것도 포함해서), `torch.no_grad()`로 코드 블록을 감쌀 수 있다. 이는 특히 model을 evaluate할 때 도움이 될 수 있다. 왜냐면 model은 trainable parameters를 `requires_grad=True`로 가지고 있을 것이지만, 실제로 model을 evaluate할 땐, gradient가 필요하지 않을 것이기 때문이다.  

<br/>

`autograd` 구현을 위해서 중요한 게 하나 더 있는데, 바로 `Function`이다.  

<br/>

Tensor와 Function은 서로 연결되어있고, acyclic graph를 이용해 build up된다. acyclic graph는 computation의 history를 완벽하게 인코딩한다. 각각의 tensor는 `.grad_fn`이라는 attribute를 가지고있다. 이 attribute는 Function을 Tensor를 생성한 함수를 참조한다. 단, user에 의해서 생성된 Tensor는 제외된다. 이때의 `grad_fun`은 `None`이다. 

<br/>

만약 derivatives를 계산하기 원한다면, 단지 Tensor에서 `.backward()`를 호출하면 된다. 만약에 Tensor가 다수의 element를 가지고있는 scalar(i.e. 하나의 element data를 가지고 있다면)라면 특별한 gradient argument가 필요하다. 여기서 gradient argument는 tensor와 shape이 맞아야한다.

In [1]:
import torch
print(torch.__version__)

1.0.0.dev20181017


`requires_grad=True`로 하는 `torch.Tensor`를 생성한다.

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


tensor operation인 `y`를 정의한다.

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y`는 tensor operation이 결과이므로 `grad_fn`이 존재한다. 이를 확인해보자

In [4]:
print(y.grad_fn)

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


`.requires_grad_( ... )`는 기존에 존재하던 Tensor의 `requires_grad` flag를 대입해 변경한다.  
input flag는 기본적으로 주어지지 않는다면 `False`이다.

In [6]:
a = torch.randn(2, 2)
b = ((a * 3) / (a - 1))
print(a.requires_grad)
print(a.grad_fn)
print(b.grad_fn, end="\n\n")

a.requires_grad_(True)
b = ((a * 3) / (a - 1))
print(a.requires_grad)
print(a.grad_fn)
print(b.grad_fn)

False
None
None

True
None


## Gradients

이제 backprop을 해보자. out은 single scalr를 포함하고 있으므로, `out.backward()`는 `out.backward(torch.tensor(1.))`과 같다.

In [7]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

out.backward()

gradient $\frac{d(out)}{dx}$를 출력해보자

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


출력 결과 ``4.5``로 구성된 Matrix값을 얻을 수 있을 것이다.  
     
출력 결과 ``out``을 *Tensor* "$o$"라고 하자. 
     
그러면 위 코드 블록은 다음과 같은 식으로 구성되어있는 것을 확인할 수 있다.

$$ o = \frac{1}{4}\sum_i z_i $$  
$$ z_i = 3(x_i+2)^2$$
    
만약, $x_i = 1$일 경우, $z_i$는 다음과 같다.  
$z_i\bigr\rvert_{x_i=1} = 27$

<br/>

따라서, gradient 식은 아래와 같으며, $x_i = 1$인 경우의 gradient는 다음과 같이 구할 수 있다.  
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$  
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

<br/>

수학적으로 vector를 갖는 function이 다음과 같이 있다고 했을 때, $\vec{y}=f(\vec{x})$  
각 $\vec{x}$에 대한 $\vec{y}$의 gradient는 아래와 같이 Jacobian matrix로 나타낼 수 있다. 

<br/>

$$
  J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)$$

<br/>
<br/>

일반적으로 이야기해서 ``torch.autograd``는 vector-Jacobian product를 계산하기 위한 엔진이다.  
vector가 $v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$와 같이 주어졌을 때, product을 다음과 같이 연산한다.  
     
$$v^{T}\cdot J$$

     
<br/>

만약에 $v$가 scalar function $l=g\left(\vec{y}\right)$의 gradient라면 $v$는 아래와 같다.  
    
$$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$$  
     
chain rule에 의하면, vector-Jacobian product는 각 $\vec{x}$에 대한 $l$의 gradient가 된다.  
    
$$
  J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)$$
   
<br/>

vector-Jacobian product의 이러한 특징은 external gradient를 non-scalar ouput을 갖는 model에 feed하기 매우 편하게 만들어준다.


이제 vector-Jacobina product의 예를 확인해보자

In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ -337.0619, -1119.0111,   468.8502], grad_fn=<MulBackward0>)


이제 `y`는 더 이상 scalar가 아니다. `torch.autograd`는 full jacobian을 직접적으로 구할 수는 없지만, vector-Jacobian product를 구하기 원한다면 vector를 bacakward의 argument로 간단하게 넘겨주면 된다.

In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

`with torch.no_grad()`구문을 통해서 `autograd`가 `.requires_gard=True`로 설정된 Tensor에서 추적하는 history 추적하지 않도록 중단할 수 있다.

In [10]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
